# LDA

In [8]:
import numpy as np
import pandas as pd

In [1]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
y = np.array([1, 1, 1, 2, 2, 2])
clf = LinearDiscriminantAnalysis()
clf.fit(X, y)
print(clf.predict([[-0.8, -1]]))


[1]


In [18]:
?? LinearDiscriminantAnalysis

In [2]:
X

array([[-1, -1],
       [-2, -1],
       [-3, -2],
       [ 1,  1],
       [ 2,  1],
       [ 3,  2]])

In [3]:
np.var(X[y == 1])

0.5555555555555556

In [7]:
X[y == 1].mean(0)

array([-2.        , -1.33333333])

In [5]:
np.unique(y)

array([1, 2])

In [2]:
def lda(x,y):
    
    c1,c2 = np.unique(y)
    
    s1 = np.var(x[y == c1])
    s2 = np.var(x[y == c2])
    
    mean_diff = x[y==c1].mean(0) - x[y==c2].mean(0)
    
    w = (mean_diff ** 2) /(s1 + s2)
    print(w.shape)
    
    return w
    
weight = lda(X,y)    

(2,)


In [23]:
weight

array([14.4,  6.4])

In [15]:
1 if np.array([-0.8, -1]).dot(weight) < 0 else 2

1

In [22]:
1 if np.array([2, 3]).dot(weight) < 0 else 2

2

# Decision Tree

In [24]:
class node:
    
    def __init__(self):
        self.value = None 
        self.right = None 
        self.left = None 
        

In [ ]:
class Tree:
    
    def __init__(self):
        pass

## Information Gain and Entropy 

In [9]:
### entropy 
### formula = -p+ log (p+) - p_ log(p_)


def entropy(y):
    
    classes = np.unique(y) # 1 2 3
    length = len(y)
#     print(classes,length)
    entropy = 0
    for label in classes:
        p = np.sum(y == label) / length
        entropy += p * np.log2(p)
        
    return -entropy

In [4]:
4 / 8 * np.log2(4/8) + 4/8 * np.log2(4/8)

-1.0

In [10]:
#checking entropy works 
y = [1,1,1,1,0,0,0,0]      

entropy(y)

1.0

In [37]:
#dataset 
dataset = {'toothed':[1,1,1,0,1,1,1,1,1,0],
#            'hair' : [1,1,0,1,1,1,0,0,1,0],
           'breathes':[1,1,1,1,1,1,0,1,1,1],
            'legs':[1,1,0,1,1,1,0,0,1,1],
           'species':[1,1,0,1,1,1,0,0,1,0]
          }

dataset  = pd.DataFrame(dataset)
dataset.head(3)

,toothed,breathes,legs,species
0,1,1,1,1
1,1,1,1,1
2,1,1,0,0


In [38]:
x = np.array(dataset)
x[x[:,1] == 1]

array([[1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 0, 0],
       [0, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 0, 0],
       [1, 1, 1, 1],
       [0, 1, 1, 0]], dtype=int64)

In [131]:
x[x[:,0] == 1][:,-1]

array([1, 1, 0, 1, 1, 0, 0, 1], dtype=int64)

In [15]:
# information gain 
#formula : Gain(X,'Feature') = entropy(X) - weighted sum entropy(X'Feature')

def info_gain(X,feature_i):
    
    entropy_X = entropy(X[:,-1])
    
    unique_values  = np.unique(X[:,feature_i]) # sunny cold winter (encoded)
    
    feature_entropy = 0
    
    
    
    for unique in unique_values:
        # get the labels of feature_i having value == unique 
        y = X[X[:,feature_i] == unique][:,-1]
        feature_entropy += entropy(y)*(len(y)/len(X))

    
    return entropy_X - feature_entropy 
    
print('info gain for feature : toothed',info_gain(x,0))    
print('info gain for feature : hair',info_gain(x,1))
print('info gain for feature : breathes',info_gain(x,2))
print('info gain for feature : legs',info_gain(x,3))


info gain for feature : toothed 0.007403392114696539
info gain for feature : hair 0.1444843438056279
info gain for feature : breathes 0.5567796494470394
info gain for feature : legs 0.9709505944546686


# Information gain and Entropy are working correctly

In [16]:
# splitting the data 

def split_dataset(X,feature_i):
    
    # for now only working on binary data.
    unique_values = np.unique(X[:,feature_i[-1]]) # 0 1 
    
    datasets = []
    
    for unique in unique_values:
        
        datasets.append(X[X[:,feature_i[-1]] == unique][:,[cols for cols in range(X.shape[1]) if cols not in feature_i]])
    
    
    # for now only considering two values 
    
    return datasets[0],datasets[1]
    
    

right,left = split_dataset(x,[1])

In [26]:
right

array([[1, 1, 0, 0],
       [1, 0, 0, 0],
       [1, 1, 0, 0],
       [0, 1, 1, 0]], dtype=int64)

In [27]:
left

array([[1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1]], dtype=int64)

# split functions work

In [17]:
# function to find feature to split on 


def split_on(X):
    
    # X has input features + concatenated output labels 
    
    no_features  = X.shape[1] - 1
    
    #iterate through all the features get the info - gain and select the one with 
    #max gain 
    
    info_gains = []
    
    for feature_i in range(no_features):
        
        info_gains.append(info_gain(X,feature_i))
    
    best_feature_i = np.argmax(info_gains)
    info_gain_i = info_gains[best_feature_i]
    
    return best_feature_i,info_gain_i
    


In [64]:
split_on(x)

(1, 0.4199730940219749)

In [68]:
split_on(left)

(0, 0.109170338675599)

# Algorithem Steps:

1. select the best_features using the `split_on` feature
2. select the split the dataset on that feature using the `split_dataset`
3. To store the nodes create a tree, each node will store the following thing 
    1. the split_dataset function returns right and left nodes 
    2. we need to store the feature name
    3. meta-data {info:gain , 'right':val,'left':val} will make prediction easy 

In [20]:
# split the best_features  using split_on feature 

print(x.shape)

feature_i,gain = split_on(x)

print('split on feature = ',[feature_i],'info. gain = ',gain)

right,left = split_dataset(x,[feature_i])

print(right.shape,left.shape)


(10, 4)
split on feature =  [2] info. gain =  0.5567796494470394
(3, 3) (7, 3)


In [97]:
class DecisionNode:
    
    def __init__(self,feature = None,
                 right_val = None,left_val = None,gain = None,
                 value = None,right=None,left=None):
        
            self.gain = gain
            self.feature = feature
            self.right_val = right_val
            self.left_val = left_val
            self.right = right
            self.left = left
            self.value = value
#             self.meta_data = meta_data
           

In [40]:
class Tree:
    
    def __init__(self,root):
        
        self.root = root
        
    def insert(self,root,right,left):
        
        if root is None:
            root = self.root
        
        if  root.right is None:
            root.right = right
        else:
            self.insert(root.right,right,left)
            
        if root.left is None:
            self.root.left = left
        else:
            self.insert(root.left,right,left)
        
        
def traverse_(node,s = 'root'):
    
    if node:
        print(node.feature,s)
        traverse_(node.left,s='l')
        traverse_(node.right,s = 'r')           

In [122]:
dataset.to_numpy().shape

array(['toothed', 'breathes', 'legs', 'species'], dtype=object)

In [148]:
class  DecisionTree:
    
    def __init__(self,max_depth,min_features):
        
        
        self.min_features = min_features
        self.max_depth = max_depth - 1
        self.root = None
        self.features = None
        
    def fit(self,X):
        
        self.features = X.columns.values
        X  = X.to_numpy()
        
        print(f'Shape of dataset :: {X.shape}')
        
        self.root  = self.build_tree(X)
        
        
    def predict(self,X,node = None):
        
        if node is None:
            node = self.root
            
        
        
    
    
    def build_tree_1(self,X):
        remove = []
        
        # create the root node then start the recursion process 
        
        ##creating the root node .
        
        feature_i,gain = split_on(X)
        
        print('root_node_feature:',feature_i,gain)
        
        right,left = np.unique(X[:,feature_i])        
        
        root_node = DecisionNode(feature_i,right,left,gain)
    
        # iterate throught the left and right of root node 
        
        tree = Tree(root_node)
        
        print(tree)
        
        
#         if self.max_depth > 1 and x.shape[1] > self.min_features:

        stack = [root_node]
    
        
        while stack!= None:
            
            node = stack.pop()
            
            # get the feature to split on 
            
            feature_i  = node.feature
            
            remove.append(feature_i)
            
            # seperate the dataset 
            
            X1,X2 = split_dataset(X,remove)
            
            
#             if X1.shape[1] == 1 or X2.shape[1] == 1:
#                 break
            
            best_feature_i,gain = split_on(X1)
            
            unique = np.unique(X1[:,best_feature_i])  
            
            if len(unique) == 1:
                break
            else:
                right,left = unique
            
            print('feature right :',best_feature_i,gain)
            print('shapes',X1.shape,X2.shape)
            
            
            right_node = DecisionNode(best_feature_i,right,left,gain)
            
            best_feature_i,gain = split_on(X2)
            
            
            unique = np.unique(X2[:,best_feature_i]) 
            
            if len(unique) == 1:
                break
            else:
                right,left = unique
            
            
            left_node = DecisionNode(best_feature_i,right,left,gain)
            
            print('feature left :',best_feature_i,gain)
            print('shapes',X1.shape,X2.shape)
            
            
            tree.insert(None,right_node,left_node)
            stack.append(right_node)
            stack.append(left_node)
            
            
        
            
        print('tree trained....') 
    
        return tree
    
    
    
    def build_tree(self,X):
     
        
        # create the root node then start the recursion process 
        
        ##creating the root node .
        
        feature_i,gain = split_on(X)
        
        
        
        unique = np.unique(X[:,feature_i])
        
        
        if len(unique) > 1:
            
             print('node :: feature=',self.features[feature_i],gain)
            
             right,left = unique
            
             X1,X2 = X[X[:,feature_i] == right],X[X[:,feature_i] == left]
                
              
              
             right_tree = self.build_tree(X1)
             left_tree = self.build_tree(X2) 
    
        
             return DecisionNode(feature_i,right = right_tree,
                                 left = left_tree,gain = gain,
                                 right_val = right,left_val = left)
        
        print('leaf :: value',unique[0])
        
        return DecisionNode(value = unique[0])
    
    
    def show_tree(self,classes = None):
        '''
            root 
           /   \
    left_tree  right_tree
    /      \       
  leaf    leaf 
        '''
        if classes:
            self.traverse_(self.root)
        else:
            self.traverse_(self.root,classes)
    
    def traverse_(self,node,s = 'root',classes = None):
        '''
        node : right,left -> this 2 attributes point to the right_sub_tree & left_sub_tree
               
               right_val , left_val : this values help to move left or right 
               
               1. detect if node has right & left : then this is node 
               2. if it doesnt have right & left and value is not null then its a leaf 
        '''
        
        #printing nodes with 
        if node.right and node.left:
            
            # feature_i right_val left_val
            print(self.features[node.feature])
            
            print(f'/{node.right_val}  \\{node.left_val}')
            if classes:
                print(f'/{classes[node.right_val]}\\{classes[node.left_val]}')
            
            if node.left.value is not None:
                 print('leaf ',node.left.value)
                 if classes:
                    print(classes[node.left.value])
                           
            if node.right.value is not None:
                print('leaf',node.right.value)
                if classes:
                     print(classes[node.right.value])
       
            self.traverse_(node.left,s='left_tree')     
            self.traverse_(node.right,s = 'right_tree')   
                  
        
    

In [149]:
model = DecisionTree(2,3)
model.fit(dataset)

Shape of dataset :: (10, 4)
node :: feature= legs 0.5567796494470394
leaf :: value 1
node :: feature= toothed 0.3059584928680418
leaf :: value 0
leaf :: value 1


In [150]:
model.show_tree()

legs
/0  \1
leaf 1
toothed
/0  \1
leaf  1
leaf 0


In [151]:
model.show_tree()

0

In [125]:
#make classification 
from sklearn import datasets

np.random.randint(1,0,)
